# Statistics in Python

*IMPRS - Using Python for Cognitive Science (2022). This tutorial is made by Noor Seijdel and based on work by [Sophie Slaats](https://www.mpi.nl/people/slaats-sophie)*

Good afternoon! Today we will be working through this tutorial to get familiar with doing statistics in Python. Important note: I am by no means a statistical advisor. I will just show you today how to use a couple of packages to do some statistical tests. Basically, I provide some pointers. When doing analyses on your own data, make your own decisions on which tests to use.
The reference guides for the packages we used in this session are here:

- [Python's statsistics](https://docs.python.org/3/library/statistics.html) 
Python’s statistics is a built-in Python library for descriptive statistics. It is not intended to be a competitor to libraries such as NumPy or SciPy or full-featured statistics packages aimed at professional statistics. It is aimed at the level of graphing and scientific calculators. 
- [Numpy](https://numpy.org/doc/stable/)
NumPy is a library for numerical computing, optimized for working with single- and multi-dimensional arrays. This library contains many routines for statistical analysis.
- [SciPy Stats](https://docs.scipy.org/doc/scipy/reference/stats.html)
SciPy is a library for scientific computing based on NumPy. It offers additional functionality compared to NumPy, including scipy.stats for statistical analysis.
- [Statsmodels](https://www.statsmodels.org/stable/index.html) 
Statsmodels is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration. It supports specifying models using R-style formulas and pandas DataFrames. 
- [Seaborn](https://seaborn.pydata.org/index.html) 
Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.
- [Pandas](https://pandas.pydata.org/)
Pandas is a library for numerical computing based on NumPy. It excels in handling labeled data with DataFrame objects.



## 0. Importing modules
Today we need `math`, `statistics`, `numpy`, `pandas` and `os`.   

<font color='green'>**Exerc|ise 1:**</font> Import them below. We have added some other packages for you already.  

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

## your code here

## 1. Loading the data + descriptives

Let's start by getting some data to work with. We can use the participant data from a couple weeks ago! 

In [ ]:
path = 'P://workspaces/nbl-communicativebrain/working_data/Noor/teaching/IMPRS_Python/'

# load the data we used previously
participants = pd.read_csv(os.path.join(path, 'participants.csv'))
participants

Okay, so that's the overview of our participants.. Now for each participant we have a separate file containing the data. Let's use a for loop to get all data in one structure. 

<font color='green'>**Exercise 2:**</font> Use a for-loop to go through the participant list and append the data from each participant to a dataframe `trials`

(Hint: we did this in the preprocessing notebook from session 5!) 


In [ ]:
# use a for-loop to go through the participant list and load the data
trials = pd.DataFrame()

## your code here 


In [ ]:
trials.head(-5)

As we've also seen before, based on their "id" we can merge the participant information with their data (note that we would actually rarely do this in real life, to keep the data anonymous). 

In [ ]:
# merge participant and trial data
trials = trials.merge(participants, on='id')
trials.rename(columns={'Unnamed: 0': 'trial_order'}, inplace=True)
trials.head(-300)

When doing data analyses you should always check your data, to make sure no weird things are in there. Quite often, your data will contain NaN values (e.g. when a participant did not press the response button after a trial, or when they did not fill in a certain question etc.). Let's simulate such a situation by adding some NaN values to our data:

In [ ]:
no_of_trials = np.shape(trials)[0]
nan_indices = np.random.permutation(no_of_trials)[0:60]
trials_NA = trials.copy()

# replace values using pd.loc: 
#function allows you to access a group of rows and columns by label(s)
trials_NA.loc[nan_indices, 'RT'] = np.nan
trials_NA.head(-5)

#### Look at the data summary

In [ ]:
summary = trials.groupby(by='condition').aggregate(
    mean_RT=pd.NamedAgg('RT', np.mean),
    median_RT=pd.NamedAgg('RT', np.median),
    std_RT=pd.NamedAgg('RT', np.std),
    mean_age=pd.NamedAgg('age', np.mean)
)

summary

In [ ]:
# reset the index (we've seen this before)
summary.reset_index(inplace=True)
summary

#### Plot the results
Now we can use seaborn to plot the results:

In [ ]:
# as we've seen in session 5:
sns.boxplot(x='condition', y='RT', data=trials)
plt.show()

<font color='green'>**Exercise 3:**</font> Try to make a violinplot of the data

In [ ]:
### your code here




If we compare these plots to the ones we made in session 5, it seems that our NaN values are dealt with appropriately (the plots are the same). Good to know: using Pandas's groupby-function handles your NaN-values for you. However, if you just want to get some means...


In [ ]:
python_stats = statistics.mean(trials_NA["RT"])
numpy_stats = np.mean(trials_NA["RT"])

print(python_stats, numpy_stats)

So when you start working with a new package or module, it is useful to first check how it deals with missing values. If you want to use python statistics, you should remove your NaN-values before computing your descriptive statistics. Fortunately there is a useful function for that: the Pandas `.dropna` function.


In [ ]:
trials_dropna = trials_NA.dropna()

<font color='green'>**Exercise 4:**</font> print the mean RT of our new `trials_dropna` 

<font color='green'>**Exercise 5:**</font> What is the mean age of our participants?

## 2. Testing assumptions

Let's say we'd like to perform a t-test. Then we first need to check the assumptions: 

1. Scale of measurement: ordinal or continuous scale
2. Simple random sample & reasonable sample size
3. Normal distribution of dependent variables
4. Homogeneity of variance


#### 2.1 Normal distribution

Graphical methods: Plotting & evaluating




In [ ]:
# Use Seaborn for a simple histogram
sns.histplot(trials, x="RT", hue="condition")

Hmm.. this is not a normal distribution; it is skewed and can be best described by an exponential. But wait - the means have to be normally distributed!


In [ ]:
pmeans = trials.groupby(by=['id', 'condition']).aggregate(
    mean_RT=pd.NamedAgg('RT', np.mean),
    std_RT=pd.NamedAgg('RT', np.std))

pmeans.reset_index(inplace=True)
pmeans.head(5)

In [ ]:
ax = sns.histplot(pmeans, x="mean_RT", hue="condition", kde=False)

<font color='green'>**Exercise 5:**</font> Try to plot the histogram with more bins (e.g. 20)

In [ ]:
### your code here


OK, another plot. Scipy.stats.probplot generates a probability plot of sample data against the quantiles of a specified theoretical distribution (the normal distribution by default). Probplot optionally calculates a best-fit line for the data:


In [ ]:
for cond in ['baseline', 'condition_a', 'condition_b']:
    plot = stats.probplot(pmeans.loc[pmeans['condition']==cond, 'mean_RT'], plot=plt)
    plt.title(str(cond))
    plt.show()

<font color='purple'>**To think:**</font> Why can we just use `stats.probplot`? Don't we need to indicate that we want to use SciPy?

Looking at these plots can be confusing. Let's try a statistical test: the Shapiro-Wilk test for normality.
Important to keep in mind: The Shapiro-Wilk test (or another test, Kolmogorov-Smirnov) is too sensitive when sample sizes are large.


In [ ]:
# Simple shapiro test:
pmeans.groupby('condition').aggregate(stats.shapiro)

In [ ]:
shapiro_results = pmeans.groupby(by='condition').aggregate(
    shapiro=pd.NamedAgg('mean_RT',stats.shapiro))

shapiro_results.reset_index(inplace=True)
shapiro_results

<font color='green'>**Exercise 6:**</font> Look at the results (and the online documentation on the test). What do you conclude?

## 3. Performing a T-test using SciPy

Okay, time to perform our first t-test to see if there are differences between our `condition_a` and the `baseline`. 
For this, we can use [scipy.stats.ttest_rel](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html):

In [ ]:
base = trials.loc[trials['condition']=='baseline', 'RT']
a = trials.loc[trials['condition']=='condition_a', 'RT']

stats.ttest_rel(base,a, nan_policy='raise')

Okay! So condition a seems very different from the baseline. 

<font color='green'>**Exercise 7:**</font> Write a for loop that performs a t-test for both our conditions and prints the result:


In [ ]:
## your code here 
# for cond in ['condition_a', 'condition_b']:


Now - do the conditions differ from each other?

<font color='green'>**Exercise 8:**</font> Perform a t-test to compare condition a and condition b: 

In [ ]:
## your code here



Of course, we should correct for multiple comparisons here. We can do this, but it would be better to use a repeated measures ANOVA. For this, we need the package Statsmodels.

## 4. Anova and LMM using Statsmodels

statsmodels is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration. An extensive list of result statistics are available for each estimator. The results are tested against existing statistical packages to ensure that they are correct.  The online documentation is hosted at [statsmodels.org](statsmodels.org).

#### Repeated Measures ANOVA



In [ ]:
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM

In the Statsmodels ANOVA example below we use our dataframe object, `trial`, as the first argument, followed by our dependent variable `RT`, subject identifier `id`, and the list of the independent variable, `condition`. At the end, we are getting the fit so that we can print the ANOVA table.

In [ ]:
aov = AnovaRM(
    trials,
    depvar='RT',
    subject='id',
    within=['condition'],
    aggregate_func='mean'
).fit()

print(aov)

#### Linear Mixed Models

In the early days, linear mixed models were not available in Python and one had save the data from Python, open up the data in R and run the model. Over the years, R & Python got to know each other a little better and several options have emerged for running LMM analyses in Python. Today we will use Statsmodels: 

https://www.statsmodels.org/stable/mixed_linear.html

The statsmodels imputation of linear mixed models (MixedLM) closely follows the approach outlined in Lindstrom and Bates (JASA 1988). This is also the approach followed in the R package LME4. Other packages such as Stata, SAS, etc. should also be consistent with this approach, as the basic techniques in this area are mostly mature.

Here we show how linear mixed models can be fit using the MixedLM procedure in statsmodels:

- Formula to specify the model. Here: RT ~ condition
- Data for the model. Here: trials
- Re_formula: one-sided formula defining the variance structure of the model (Default = random intercept for each group). Here: 1
- Groups: random intercept



The outcome variable is the RT, and the only predictor variable we will use here is “condition”. First we fit a model that expresses the mean RT as a function of condition, with a random intercept for each participant. The model is specified using formulas. Since the random effects structure is not specified, the default random effects structure (a random intercept for each group) is automatically used:

In [ ]:
import statsmodels.formula.api as smf

lmm0 = smf.mixedlm("RT ~ condition", trials, groups = 'id', re_formula='1')
lmm0f = lmm0.fit()
print(lmm0f.summary())

We can add more predictor variables by editing our formula: 

In [ ]:
lmm1 = smf.mixedlm("RT ~ condition + age", trials, groups = 'id', re_formula='1')
lmm1f = lmm1.fit()
print(lmm1f.summary())

In [ ]:
lmm2 = smf.mixedlm("RT ~ condition + age + condition * age", trials, groups = 'id', re_formula='1')
lmm2f = lmm2.fit()
print(lmm2f.summary())

In [ ]:
lmm3 = smf.mixedlm("RT ~ condition + condition * age", trials, groups = 'id', re_formula='1')
lmm3f = lmm3.fit()
print(lmm3f.summary())



<font color='pink'>**Homework exercise**</font>

This is our final session. After today, you know how to work with Python on your own laptop, to work with GIT for code organisation (and version control), to use Python's built-in data types, functions and modules to solve tasks like renaming files or generating random lists of experimental stimuli. On top of all that, you know how to use external packages (and more importantly to solve weird annoying errors installing external packages), program a simple experiment and how to import, process and analyze experimental data. 

Now let's put that into practice! For this final homework assignment we like you to read in a dataset of choice. We ask you to create an analysis-notebook, that should contain at least:

- A nice introduction at the top, explaining what you did (in a Markdown cell)
- A table summarizing your data
- A plot of your contrast of interest
- A statistical test to determine its significance
- In between: text boxes in markdown describing what you are doing

We like you to do this in a completely new, fresh notebook.


Here we'd also like to thank you for your great efforts (and for not getting mad when we encountered so many installation errors). We really had a great time and hope you learned something useful :) . 